In [ ]:
# My purpose with this notebook is to have a solid yet simple approach for this challenge (even tough the competition is not open anymore). There are several comments in the code to help clarify the steps I'm taking. I hope this is useful for you. If you have any questions, please let me know. I'll be happy to help.

In [ ]:
%%writefile submission.py

# Imports helper functions
from kaggle_environments.envs.halite.helpers import *
import random
import math

DEBUG=False
convert_cost = 500
spawn_cost = 500
halite_min_default = max(convert_cost, spawn_cost) # 500
# When the ship has more than this amount, it will go to the closest shipyard
collect_max = halite_min_default # 500
# The minimum amount of halite to deposit in a shipyard
# (will try to get more until collect_max is reached,
# but may deposit first due to enemy ships or having
# to go collect elsewhere when it would be better to
# deposit first; in some cases, it may deposit even
# when it has less then this amount due to enemies)
collect_min = collect_max // 10 # 50
# When the cell in which the ship is located has less than this amount,
# go to another cell, or deposit (if the ship has at least collect_min
# halite and the deposit has a good halite_per_step ratio)
cell_min_halite = collect_max // 10 # 50
# List of tupples. For each tuple, the first element is how many steps
# the game is going on, and the second is a multiplier. If the current
# step is less than the defined step, use the multiplier to define the
# maximum number of ships, otherwise verify the next element (elements
# to the right must have a higher number of steps). The maximum number
# of ships will be defined as multiplier*size, rounded up, where size is
# the size of the board. If no condition is satisfied, will define the
# maximum amount of ships as 1 (the absolute amount, not the multiplier).
max_ships_steps = [(250, 2), (300, 1.5), (350, 1), (375, 0.5), (390, 0.2), (395, 0.1)]

def debug(*args):
    if DEBUG:
        print(*args)

def dirs_to(from_pos, to_pos, size):
    #Get the actions you should take to go from Point from_pos to Point to_pos
    #using shortest direction by wraparound
    #Args: from_pos: from Point
    #      to_pos: to Point
    #      size:  size of board
    #returns: list of directions, tuple (deltaX,deltaY)
    #The list is of length 1 or 2 giving possible directions to go, e.g.
    #to go North-East, it would return [ShipAction.NORTH, ShipAction.EAST], because
    #you could use either of those first to go North-East.
    #[] is returned if from_pos==to_pos and there is no need to move at all
    deltaX, deltaY = to_pos - from_pos

    if abs(deltaX) > size / 2:
        #we wrap around
        if deltaX < 0:
            deltaX += size
        elif deltaX > 0:
            deltaX -= size

    if abs(deltaY) > size / 2:
        #we wrap around
        if deltaY < 0:
            deltaY += size
        elif deltaY > 0:
            deltaY -= size

    actions_d = []

    if deltaX > 0:
        actions_d.append((ShipAction.EAST, deltaX))
    if deltaX < 0:
        actions_d.append((ShipAction.WEST, deltaX))
    if deltaY > 0:
        actions_d.append((ShipAction.NORTH, deltaY))
    if deltaY < 0:
        actions_d.append((ShipAction.SOUTH, deltaY))

    # Actions with greater absolute distance first. This makes the movement
    # go "diagonally", when it can, which increases the number of possible
    # directions during a considerable part of the trajectory
    # (if the ship goes in only one direction first, then, from then on, it
    # must go only in one direction until the target, which makes it more
    # difficult to avoid enemy ships)
    actions_d = sorted(actions_d, reverse=True, key=lambda ad: abs(ad[1]))
    actions = [a for a, d in actions_d]

    return actions, (deltaX, deltaY)

# Returns best direction to move from one position (from_pos) to another (to_pos)
def get_dir_to(from_pos, to_pos, size, allowed_directions):
    directions, _ = dirs_to(from_pos, to_pos, size)
    directions = [d for d in directions if d in allowed_directions]
    direction = directions[0] if directions else None
    return direction

# Returns the distance between 2 positions
def get_distance(from_pos, to_pos, size):
    _, deltas = dirs_to(from_pos, to_pos, size)
    delta_x, delta_y = deltas
    return abs(delta_x) + abs(delta_y)

# Returns the sum of the distances between a position and a list of other positions
def total_distance(position, positions, size):
    return sum([get_distance(position, p, size) for p in positions])

# Returns the sum of the distances between a position and all the ships specified
def distance_from_ships(position, ships, size):
    return total_distance(position, [ship.position for ship in ships], size)

# Returns the sum of the distances between a position and all the shipyards specified
def distance_from_shipyards(position, shipyards, size):
    return total_distance(position, [shipyard.position for shipyard in shipyards], size)

# Returns the closest shipyard, as long as at least one of the directions to go is allowed
def get_closest_shipyard(position, shipyards, allowed_directions, size):
    # get closest shipyards first
    closest_shipyards = sorted(shipyards, key=lambda s: get_distance(position, s.position, size))
    closest_shipyard = None
    direction = None

    for shipyard in closest_shipyards:
        direction = get_dir_to(position, shipyard.position, size, allowed_directions)

        if direction:
            closest_shipyard = shipyard
            break

    return closest_shipyard, direction

# Amount of halite per step
# https://www.kaggle.com/code/solverworld/optimal-mining-with-carried-halite
def R(n1, n2, m, H):
    return (1-.75**m)*H/(n1+n2+m)

# Evaluate the cost of going to a cell, collecting until the ship halite plust the collected
# halite exceeds the amount to halite to move to the shipyard ({collect_max}), or until the
# amount of halite in the cell becomes less than {cell_min_halite}, and then returning to the
# closes shipyard (the higher the cost, the worse)
def evaluate_cost(ship, cell, shipyards, allowed_directions, size):
    shipyard, _ = get_closest_shipyard(cell.position, shipyards, allowed_directions, size)

    n1 = get_distance(ship.position, cell.position, size)
    n2 = get_distance(cell.position, shipyard.position, size) if shipyard else 1
    H = cell.halite
    halite_per_step = max([R(n1, n2, m, H) for m in range(1, 4)])

    halite_per_step = 0

    # Iterate until m (number of turns to mine) reaches 10, or end before if:
    # - The collected halite in m turns plus the current halite in the ship reaches collect_max; or
    # - The remaining halite in the cell becomes less than cell_min_halite
    for m in range(1, 10):
        halite_per_step = R(n1, n2, m, H)
        remaining_halite = (.75**m)*H
        halite_to_collect = H - remaining_halite

        if ship.halite + halite_to_collect >= collect_max:
            break

        if remaining_halite < cell_min_halite:
            break

    # It's negative because more halite per step equals to less cost
    return -halite_per_step

# If the cell has less than {cell_min_halite} halite, go to another cell to collect
def go_to_collect(ship, board, allowed_directions, not_allowed_positions, should_move, size):
    if not allowed_directions:
        return None, None

    all_cells = board.cells.values()
    me = board.current_player
    shipyards = me.shipyards
    direction, final_pos = None, None
    cost = None

    if should_move or (ship.cell.halite < cell_min_halite):
        cells = [
            cell for cell in all_cells
            if cell.halite >= cell_min_halite and cell.position not in not_allowed_positions
        ]

        if cells:
            # choose the cell with better cost-benefit, prioritizing closer cells, and cells with more halite
            ordered_cells = sorted(cells, key=lambda cell: evaluate_cost(ship, cell, shipyards, allowed_directions, size))

            for cell in ordered_cells:
                direction = get_dir_to(ship.position, cell.position, size, allowed_directions)
                if direction:
                    final_pos = cell.position
                    cost = evaluate_cost(ship, cell, shipyards, allowed_directions, size)
                    break

            if direction:
                debug(title('collect'), ship.id, cell.halite, final_pos, direction)

        # If the ship should move (due to an enemy) or should go collect (due to a small amount of halite in the cell)
        # and it has at least collect_min halite, it may consider going to deposit in a shipyard instead, if the cost
        # of going there is not higher then the cost of collecting elsewhere (that is, it will go to the shipyard
        # if the halite_per_step of going there is equal or higher than going to collect more in another cell)
        # It's also important to deposit so as to care less about enemies and having less chance of being destroyed.
        if ship.halite and (should_move or (ship.halite >= collect_min)):
            shipyards_free = [s for s in shipyards if s.position not in not_allowed_positions]
            shipyard, direction_sy = get_closest_shipyard(ship.position, shipyards_free, allowed_directions, size)

            if shipyard and direction_sy:
                steps = get_distance(ship.position, shipyard.position, size)
                halite_per_step = ship.halite/steps if steps else 0
                cost_sy = -halite_per_step
                # Deposit if the halite_per_step of going to the shipyard is equal or higher than going to collect
                # more in another cell, or if there's no cell to go collect (cost is None)
                if steps and (cost_sy <= (cost or 0)):
                    direction = direction_sy
                    final_pos = shipyard.position

    return direction, final_pos

def title(name):
    return f'[{name:8}]'

# Dictionary (Dict[ShipId, Point]) associating ships with positions
# The values are kept during several turns until the ship reaches
# the position, or ceases to exist
positions_to_go = {}

# Returns the commands we send to our ships and shipyards
def agent(obs, config):
    size = config.size
    board = Board(obs, config)
    me = board.current_player

    max_ships_mult = ([m for s, m in max_ships_steps if board.step < s] + [1])[0]
    max_ships = math.ceil(max_ships_mult*size)
    # Will try to keep shipyards as 1/4 of the number of ships, with a minimum
    # of 1 (as long as there's a ship)
    min_shipyards = min(len(me.ships), max(((len(me.ships) - 4) // 4) + 1, 1))
    # minimum halite to keep when spawning a ship
    halite_min = 0 if len(me.ships) == 0 else (halite_min_default if len(me.ships) < size else 2*halite_min_default)
    close_to_end = board.step >= 390
    ending = board.step >= 398

    # positions in which ships are going to move to in this turn
    positions_moved_to = []
    # positions in which ships moved from in this turn
    positions_moved_from = []
    my_shipyards_positions = [s.position for s in me.shipyards]
    enemy_ships = [s for s in board.ships.values() if s.player_id != me.id]
    enemy_shipyards = [s for s in board.shipyards.values() if s.player_id != me.id]

    # Remove persistent positions with the ship ids specified
    del_pos_ship_ids = [
        ship_id for ship_id in positions_to_go
        if (ship_id not in board.ships) or (board.ships[ship_id].position == positions_to_go[ship_id])
    ]
    for ship_id in del_pos_ship_ids:
        del positions_to_go[ship_id]

    # Make sure to convert ships to shipyards until the amount is equal to min_shipyards,
    # as long as the ships and/or the player have the amount of halite required to convert
    # At most half of the ships (rounded up) can be converted in a turn, to avoid losing too many ships
    # (convert change a ship into a shipyard, so the division by 2 is done to define the maximum)
    if len(me.shipyards) < min_shipyards:
        # list the ships when the player halite plus the ship halite is greater than the cost of
        # converting a ship, and then spawning a ship
        ships = [s for s in me.ships if (me.halite + s.halite) >= (convert_cost + spawn_cost)]
        maximum = max(min_shipyards - len(me.shipyards), 0)
        maximum = min((len(ships) + 1) // 2, maximum)

        if ships and maximum:
            # create a list with the ships more distant from existing shipyards first
            # to prioritize converting ships more distant from existing shipyards
            ordered = sorted(
                ships,
                reverse=True,
                key=lambda s: distance_from_shipyards(s.position, me.shipyards, size)
            )[:maximum]

            for ship in ordered:
                debug(title('convert'), ship.id)
                ship.next_action = ShipAction.CONVERT
                positions_moved_from.append(ship.position)

    # If the player has at least halite_min + spawn_cost halite, spawn ships while the player will
    # end up with at least halite_min halite
    # (which will also create more shipyards in the long run, because of the condition above)
    # This happens as long as the number of ships stay at most equal to max_ships
    if (not close_to_end) and (me.halite >= halite_min + spawn_cost):
        shipyards = [shipyard for shipyard in me.shipyards if shipyard.next_action == None]
        maximum = max(min((me.halite - halite_min) // spawn_cost, max_ships - len(me.ships)), 0)

        if shipyards and maximum:
            # create a list with the shipyards more distant from existing ships first
            # to prioritize spawning ships more distant from existing ships
            ordered = sorted(
                shipyards,
                reverse=True,
                key=lambda s: distance_from_ships(s.position, me.ships, size)
            )[:maximum]

            for shipyard in ordered:
                shipyard.next_action = ShipyardAction.SPAWN
                positions_moved_to.append(shipyard.position)

    # Ships in shipyards first, followed by ships with more halite
    ships = sorted(me.ships, reverse=True, key=lambda s: 2*collect_max if s.cell.shipyard else s.halite)

    for ship in ships:
        if ending and ship.halite > convert_cost:
            ship.next_action = ShipAction.CONVERT
            debug(title('convert'), 'end', ship.id)
        elif ship.next_action == None:
            directions = [ShipAction.NORTH, ShipAction.EAST, ShipAction.SOUTH, ShipAction.WEST]
            neighbor_cells = [ship.cell.north, ship.cell.east, ship.cell.south, ship.cell.west]
            neighbor_positions = [cell.position for cell in neighbor_cells]
            my_other_ships_positions = positions_moved_to + [
                s.position for s in me.ships
                if s.id != ship.id and s.position not in positions_moved_from
            ]
            enemies_to_avoid = [s for s in enemy_ships if s.halite <= ship.halite]
            enemy_danger_positions = [
                position
                for s in enemies_to_avoid
                for position in [s.cell.north.position, s.cell.east.position, s.cell.south.position, s.cell.west.position]
                if position not in my_shipyards_positions
            ]
            enemy_positions = (
                [s.position for s in enemies_to_avoid]
                + [s.position for s in enemy_shipyards]
                + enemy_danger_positions)
            positions_to_avoid = enemy_positions + my_other_ships_positions
            allowed_cells = [
                cell for cell in neighbor_cells
                if cell.position not in positions_to_avoid
            ]
            allowed_positions = [cell.position for cell in allowed_cells]
            allowed_directions = [directions[neighbor_positions.index(pos)] for pos in allowed_positions]
            should_move = ship.position in enemy_positions

            if not allowed_positions:
                # Stay in position if it's safe, otherwise try to convert if there's available halite
                if should_move:
                    convert = (ship.halite >= convert_cost) or (ship.halite + me.halite >= 2*halite_min_default)
                    debug(title('unsafe'), 'convert' if convert else 'stay', ship.id, ' - ', ship.halite, me.halite)
                    # If the ship can convert, do it, else do nothing
                    if convert:
                        ship.next_action = ShipAction.CONVERT
            else:
                direction = None

                # If cargo gets very big, deposit halite
                if len(me.shipyards) and ((ship.halite >= collect_max) or (close_to_end and (ship.halite >= collect_min))):
                    shipyard, direction = get_closest_shipyard(ship.position, me.shipyards, allowed_directions, size)
                    if shipyard and direction and shipyard.position == neighbor_positions[directions.index(direction)]:
                        debug(title('shipyard'), ship.id, ship.halite, ' - ', ship.position, shipyard.position, direction)

                if positions_to_go.get(ship.id) and not direction:
                    # Go to position already defined in a previous turn to collect halite, if possible
                    direction = get_dir_to(ship.position, positions_to_go[ship.id], size, allowed_directions)

                if not direction:
                    # Move to collect halite if the current cell has less then the minimum expected;
                    # persist the position to use in the next turns
                    not_allowed_positions = list(positions_to_go.values())
                    direction, final_pos = go_to_collect(ship, board, allowed_directions, not_allowed_positions, should_move, size)
                    positions_to_go[ship.id] = final_pos

                if should_move and not direction:
                    # Go to a random safe position
                    idx = neighbor_positions.index(random.choice(allowed_positions))
                    direction = directions[idx]
                    debug(title('random'), ship.id)

                if direction:
                    positions_moved_to.append(neighbor_positions[directions.index(direction)])
                    positions_moved_from.append(ship.position)
                    ship.next_action = direction

    if board.step < 10 or board.step % 10 == 0:
        print(
            f'[{board.step}]',
            '>> me <<',
            f'| halite={me.halite} |',
            f'| ships={len(me.ships)} |',
            f'| shipyards={len(me.shipyards)} |'
        )
    return me.next_actions

In [ ]:
# Simple Yet Effective Halite Bot Strategy

The code cell below outlines a basic yet functional strategy for a Halite game bot. While it may not represent the pinnacle of strategic complexity or optimization (you can see more powerful implementations in some of the notebooks provided by other kagglers), it embodies a straightforward approach to navigating the game's challenges. The simplicity of the implementation makes it accessible and easy to understand, serving as a solid foundation for further enhancements.

That is, my purpose with this notebook was to have a solid yet simple approach for this challenge (even tough the competition is not open anymore).

## Importing Necessary Libraries

The code begins by importing essential functions from the Halite game environment and other standard libraries such as random and math.

I avoided more specific and complex libraries to keep the code simple and easier to understand.

## Debugging Setup

A simple debugging mechanism is established through a DEBUG flag and a debug function. This setup allows for conditional printing of debug information, aiding in the development and troubleshooting process without cluttering the output during normal execution.

## Ship Management Strategy

The `max_ships_steps` list outlines a dynamic ship management strategy, adjusting the maximum number of ships based on the game's progress. This adaptive approach considers both the game's duration and the board size, aiming to optimize the fleet size for different stages of the game.

## Target positions

The `positions_to_go` dictionary is used to map ships to their target positions. This is not kept in the agent function so that the data kept in it can be persisted during several turns.

Here's a breakdown of its uses and functionality:

* Tracking Movement: It keeps track of where each ship is supposed to go. By associating each ship with a position, the algorithm can easily determine the next move for each ship. The implementation was done to allow defining a position to a ship only if there's no ship going to this position yet.

* Turn-based Updates: The positions are maintained over several turns, and ships move towards their target positions incrementally. After each turn, the ships' positions are updated based on their current location and their target destination in positions_to_go.

* Dynamic Updates: The dictionary is updated dynamically. If a ship reaches its target position, the corresponding entry is removed. Similarly, if a ship is destroyed or removed from the game, its entry is removed from the dictionary.

If there's no available direction that the ship can go to stay closer to the target position (due to other ships, shipyards or enemies), the ship will try to define and go to another position.

## The Agent

The agent function is designed to generate commands for ships and shipyards in a game environment, based on the current game observation (obs) and game configuration (config). Here's a step-by-step explanation of what happens inside the agent function:

* Initialize Game Board: It creates a Board object using the current game observation and configuration. This object represents the game state, including all ships, shipyards, and their positions.

* Player Identification: It identifies the current player (`me`) by accessing `board.current_player`. This is used to make decisions specific to the player's assets (ships and shipyards).

* Max Ships Calculation: It calculates the maximum number of ships (`max_ships`) the player should aim for. This is based on a multiplier (`max_ships_mult`) determined by the current game step compared to predefined steps and multipliers in `max_ships_steps`. The idea is to dynamically adjust the target number of ships as the game progresses.

* Min Shipyards Calculation: It calculates the minimum number of shipyards (`min_shipyards`) the player should maintain. This is based on the current number of ships, aiming to keep shipyards as 1/4 of the number of ships, with at least one shipyard as long as there is at least one ship (to convert).

* Halite Management: It determines the minimum halite (`halite_min`) to keep when spawning a ship. This threshold is adjusted based on the number of ships the player has, encouraging more conservative halite spending as the player's fleet grows.

* End Game Detection: It checks if the game is close to ending (`close_to_end`) or is in the final steps (`ending`), based on the current game step. This can influence more aggressive or conservative strategies as the game wraps up (when the game is close to end, the ships will try to deposit in shipyards, or convert).

* Movement Tracking: It initializes lists to track positions ships will move to (`positions_moved_to`) and from (`positions_moved_from`) during this turn. This helps in avoiding collisions and managing ship movements efficiently.

* Shipyards and Enemy Tracking: It compiles lists of the player's shipyard positions (`my_shipyards_positions`) and identifies enemy ships (`enemy_ships`) and shipyards (`enemy_shipyards`). This information is useful to determine where the ship can or can't go (`allowed_positions` and `allowed_directions`) and also if the ship shouldn't stay where it is (`should_move`).

The agent function then proceeds with the following main steps, in this order, for ship management:

* Ship Conversion: It converts ships to shipyards if the number of shipyards is less than the minimum expected. This prioritizes converting ships that are far from existing shipyards and have sufficient halite to convert and spawn a new ship.

* Ship Spawning: It spawns new ships from existing shipyards if the number of ships is less than the minimum expected. This prioritizes spawning ships that are far from existing ships.

* Deposit Halite: It checks if the ship should deposit halite in a shipyard based on the cargo size and proximity to the closest shipyard. This is important to prevent losing halite to enemy ships and to ensure efficient resource collection.

* For each ship, it determines the allowed directions based on the neighboring cells' positions and the positions to avoid (enemy ships with less halite, shipyards, and other ships' positions, including possible positions that enemy ships can move to). The ship will try to choose one of the following actions, in this order:

    * Ship Movement: If there's already a position defined for the ship, the ship will try to go in this direction.

    * If the ship should move (due to enemy presence) or should go collect more (due to low halite in the cell), it will try to find a cell with more halite to collect, or a close shipyard to deposit the cargo (it defines where to go based on the amount of halite per step of each possible action). In this case, the position is persisted across turns.

    * If no direction was defined in the previous conditions, and the ship should move and has at least 1 direction that it can go, it will move randomly.

